In [1]:
# general imports
import time
import os

# redefine system path to import outside packages...
import sys
sys.path.append('../')

# import internal tools
from src import enrichment
from src import fetch
from src import tools

In [2]:
# parameters for our search...
SUB  = 'uva' # subreddit to scrape from
TIME = 'week'         # time range
SORT = 'top'         # category by which to sort by
N_POSTS    = 1       # number of posts
N_COMMENTS = 100     # number of comments PER post
DEPTH      = 5       # comment depth (or children) limit

In [3]:
# fetch submissions
submissions = fetch.forest_fetch(
    sub  = SUB,
    time = TIME,
    sort = SORT,
    num_posts = N_POSTS,
    num_comments = N_COMMENTS,
    depth = DEPTH
)

fetching comments from uva...           
sorting by: top, scraping 1 posts, 100 comments each (depth=5)
connecting to reddit client...
connected!


In [4]:
tools.save_json(submissions, '../data/testdepth.json')